# Popularity of names analysis

### Data wrangling
Taking the [SSN names data](https://www.ssa.gov/oact/babynames/limits.html) as a series of folders and files, and parsing it into a dataframe. Primary pre-processing work.

In [2]:
# import packages
import os, shutil
import pandas as pd, numpy as np
 
# clear warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# view national names
national_folders = os.listdir('./nationalnames')
if '.DS_Store' in national_folders:
    national_folders.remove('.DS_Store')
national_folders[0:20]

['yob2000.txt',
 'yob2014.txt',
 'yob1938.txt',
 'yob1910.txt',
 'yob1904.txt',
 'yob1905.txt',
 'yob1911.txt',
 'yob1939.txt',
 'yob2015.txt',
 'yob2001.txt',
 'yob2017.txt',
 'yob2003.txt',
 'yob1907.txt',
 'yob1913.txt',
 'yob1898.txt',
 'yob1899.txt',
 'yob1912.txt',
 'yob1906.txt',
 'yob2002.txt',
 'yob2016.txt']

In [4]:
# view state names
state_files = os.listdir('./namesbystate')
if '.DS_Store' in state_files:
    state_files.remove('.DS_Store')
state_files[0:10]

['IN.TXT',
 'IL.TXT',
 'KS.TXT',
 'SC.TXT',
 'HI.TXT',
 'GA.TXT',
 'SD.TXT',
 'CO.TXT',
 'NH.TXT',
 'MS.TXT']

### Data storage form
These functions will parse through all of the non-PDF filetypes within the folder, storing them in a dataframe that has entries for year, name, sex (can have ambiguous gender of names). Those will end up being our three indexes (in reverse order - name as most important) because there shouldn't duplicates of that. Since they are both indexes, we will store the dataframe with a hierarchial index.

In [5]:
# mapper function
def national_text_df(file):
    if (str.upper(file[-4:]) != '.TXT'):
        pass
    else:
        year = int(file[-8:-4])
        file_data = pd.read_csv('./nationalnames/{}'.format(file), header = None)
        file_data.rename(dict(zip([0, 1, 2], ['name', 'sex', 'count'])), axis = 1, inplace = True)
        file_data['y'] = year
        return file_data.set_index(['name', 'y', 'sex'])

In [7]:
# passing in national data - slow runtime
national_names = pd.concat([national_text_df(i) for i in national_folders])

# sort indexes by name alphabetically, year in descending order, M/F by Python default
national_names.sort_index(axis = 0, ascending = [True, False, True], kind = 'mergesort', inplace = True)

# view head of data
national_names.head(20)

count
name      y    sex       
Aaban     2017 M       11
          2016 M        9
          2015 M       15
          2014 M       16
          2013 M       14
          2012 M       11
          2011 M       11
          2010 M        9
          2009 M        6
          2007 M        5
Aabha     2016 F        7
          2015 F        7
          2014 F        9
          2012 F        5
          2011 F        7
Aabid     2016 M        5
          2003 M        5
Aabir     2016 M        5
Aabriella 2017 F        6
          2016 F       11

In [34]:
# mapper function
def state_txt_df(file):
    if (str.upper(file[-4:]) != '.TXT'):
        pass
    else:
        state = file[:2]
        file_data = pd.read_csv('./namesbystate/{}'.format(file), header = None)[[1, 2, 3, 4]]
        file_data.rename(dict(zip([1, 2, 3, 4], ['sex', 'y', 'name', state])), axis = 1, inplace = True)
        file_data.set_index(['name', 'y', 'sex'], inplace = True)
        return file_data

In [50]:
# passing in state data - slow runtime
state_names = pd.concat([state_txt_df(i) for i in state_files], axis = 1, ignore_index = False)

# sort indexes by name alphabetically, year in descending order, M/F by Python default
state_names.sort_index(axis = 0, ascending = [True, False, True], kind = 'mergesort', inplace = True)

# fill na with 0
state_names.fillna(value = 0, axis = 1, inplace = True)
state_names = state_names.applymap(lambda x: int(x))

# view head of data
state_names.head(20)

IN  IL  KS  SC  HI  GA  SD  CO  NH  MS ...  MI  NE  KY  ID  \
name    y    sex                                         ...                   
Aaban   2014 M     0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   
        2013 M     0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   
Aadan   2014 M     0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   
        2009 M     0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   
        2008 M     0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   
Aadarsh 2009 M     0   5   0   0   0   0   0   0   0   0 ...   0   0   0   0   
Aaden   2017 M     0  17   5   5   0   9   0   0   0   0 ...   6   0   0   0   
        2016 M     0  10   5   5   0  10   0   0   0   0 ...   0   0   0   0   
        2015 M     0  12   0   8   0  16   0   5   0   5 ...  12   0   5   0   
        2014 M     0  11   0   6   0  10   0   6   0  11 ...   5   0   0   0   
        2013 M     0  11   0   5   0  10   0   0   0   6 ...   5   0   0   0   
        2012 M     0   7   0   0   0   6   0   0   0   0 ...   5   0   6   0   
        2011 M     9  10   0   6   0  10   0   7   0   0 ...   7   0   5   0   
        2010 M     8  13   6   6   0  13   0   9   0   6 ...  15   0   8   0   
        2009 M    19  65  13  11   0  39   0  22   0  18 ...  41   9  24   0   
        2008 M    18  40  10  15   6  29   6  10   0  13 ...  29   7  24   0   
        2007 M     0   8   0   0   0   8   0   6   0   0 ...   0   0   0   0   
        2005 M     0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   
Aadhav  2017 M     0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   
        2016 M     0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   

                  DC  IA  FL  PA  RI  DE  
name    y    sex                          
Aaban   2014 M     0   0   0   0   0   0  
        2013 M     0   0   0   0   0   0  
Aadan   2014 M     0   0   0   0   0   0  
        2009 M     0   0   0   0   0   0  
        2008 M     0   0   0   0   0   0  
Aadarsh 2009 M     0   0   0   0   0   0  
Aaden   2017 M     0   0  17   6   0   0  
        2016 M     0   0  14   0   0   0  
        2015 M     0   0  22   0   0   0  
        2014 M     0   0  14   6   0   0  
        2013 M     0   0  20   7   0   0  
        2012 M     0   0  19   0   0   0  
        2011 M     0   7  11   0   0   0  
        2010 M     0   0  28  18   0   0  
        2009 M     0  20  62  54   6   5  
        2008 M     0  18  51  35   0   0  
        2007 M     0   0   0   5   0   0  
        2005 M     0   0   0   0   0   0  
Aadhav  2017 M     0   0   0   0   0   0  
        2016 M     0   0   0   0   0   0  

[20 rows x 51 columns]

### Creating aggregate dataframe
For the same of speed we will organize and combine the entire dataframe, before exporting it for faster future use.

In [88]:
# ordering state columns
statenames_ordered = list(state_names)
statenames_ordered.sort()
state_names = state_names[statenames_ordered]
state_names.head()

# organizing national names - rename
national_names.rename({'count': 'national'}, axis = 1, inplace = True)

# concat
FULL_NAMES = pd.concat([national_names, state_names], axis = 1, ignore_index = False)
FULL_NAMES.fillna(value = 0, axis = 1, inplace = True)
FULL_NAMES = FULL_NAMES.applymap(lambda x: int(x))
FULL_NAMES.head()

national  AK  AL  AR  AZ  CA  CO  CT  DC  DE ...  SD  TN  TX  \
name  y    sex                                               ...               
Aaban 2007 M           5   0   0   0   0   0   0   0   0   0 ...   0   0   0   
      2009 M           6   0   0   0   0   0   0   0   0   0 ...   0   0   0   
      2010 M           9   0   0   0   0   0   0   0   0   0 ...   0   0   0   
      2011 M          11   0   0   0   0   0   0   0   0   0 ...   0   0   0   
      2012 M          11   0   0   0   0   0   0   0   0   0 ...   0   0   0   

                UT  VA  VT  WA  WI  WV  WY  
name  y    sex                              
Aaban 2007 M     0   0   0   0   0   0   0  
      2009 M     0   0   0   0   0   0   0  
      2010 M     0   0   0   0   0   0   0  
      2011 M     0   0   0   0   0   0   0  
      2012 M     0   0   0   0   0   0   0  

[5 rows x 52 columns]

### Exporting data file
Previously I haven't really worked in separate notebooks; given the size of this file though/time it takes to process, I'm trying to separate workflows in this project.

In [95]:
# export data as csv
FULL_NAMES.to_csv('babynames-data.csv')